# 01_EDA v0.1: Análisis Exploratorio de Datos (Robusto)

Este notebook realiza un análisis exploratorio exhaustivo de los datos disponibles para el proyecto de detección de fraude financiero. 

**Objetivos de esta versión 0.1**:
1. Cargar y revisar todos los archivos de `data/raw/` (o `data/backup/` si los ligeros no existen en raw).  
2. Normalizar tipos (fechas, montos, flags “Yes/No” → 1/0) y revisar datos faltantes.  
3. Combinar transacciones con etiquetas de forma correcta (según la estructura real de `train_fraud_labels.json`).  
4. Detectar duplicados, outliers y valores inconsistentes.  
5. Explorar:
   - Distribución de montos (`amount`).  
   - Balance de la variable objetivo (`target`).  
   - Fraude por estado y ciudad.  
   - Fraude por hora del día y día de la semana.  
   - Fraude por tipo de tarjeta.  
   - Top MCCs donde hay más fraudes.  
6. Calcular algunas correlaciones básicas y visualizar mapa de calor.  
7. Hacer un pequeño preprocesamiento (generar columna `hour`, flags binarias, parsear fechas).  
8. Guardar el dataset combinado limpio en `data/processed/` para la fase de modelado.

In [25]:
# 2.1 Importar librerías esenciales
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# 2.2 Estilo para gráficos
%matplotlib inline
plt.style.use('default')

# 2.3 Opciones de pandas para visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

# 2.4 Función auxiliar para inspeccionar cualquier DataFrame
def inspeccionar_df(df, nombre):
    """
    Imprime info básica: shape, dtypes, nulos y duplicados de un DataFrame.
    """
    print(f"\n--- Inspección de {nombre} ---")
    print("Dimensiones:", df.shape)
    print("\nTipos de columnas:\n", df.dtypes)
    print("\nValores faltantes por columna:\n", df.isnull().sum().sort_values(ascending=False))
    print(f"\nDuplicados exactos (filas idénticas): {df.duplicated().sum()}")
    if "transaction_id" in df.columns:
        print("Duplicados en 'transaction_id':", df["transaction_id"].duplicated().sum())
    print("-" * 60 + "\n")

# 2.5 Semilla para reproducibilidad (opcional)
import random
random.seed(42)


In [26]:
# 3.2.1 Cargar cards_data.csv
path_cards = "../data/backup/cards_data.csv"
df_cards = pd.read_csv(path_cards)

# 3.2.2 Cargar users_data.csv
path_users = "../data/backup/users_data.csv"
df_users = pd.read_csv(path_users)

# 3.2.3 Cargar mcc_codes.json, convertir a DataFrame de dos columnas
path_mcc = "../data/backup/mcc_codes.json"
with open(path_mcc, "r") as f:
    mcc_dict = json.load(f)
df_mcc = pd.DataFrame(list(mcc_dict.items()), columns=["mcc_code", "description"])

# 3.2.4 Mostrar inspección básica
inspeccionar_df(df_cards, "df_cards")
inspeccionar_df(df_users, "df_users")
inspeccionar_df(df_mcc, "df_mcc")



--- Inspección de df_cards ---
Dimensiones: (6146, 13)

Tipos de columnas:
 id                        int64
client_id                 int64
card_brand               object
card_type                object
card_number               int64
expires                  object
cvv                       int64
has_chip                 object
num_cards_issued          int64
credit_limit             object
acct_open_date           object
year_pin_last_changed     int64
card_on_dark_web         object
dtype: object

Valores faltantes por columna:
 id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
dtype: int64

Duplicados exactos (filas idénticas): 0
-------------------------------------------------------